# Portfolio Optimization (포트폴리오 최적화)

- 위험회피적 투자자가 시장 위험의 주어진 수준에 따라 기대수익을 최적화하거나 최대화하여 보상 수준을 선택  


- 방법 1 – Monte Carlo Simulation 에 의한 최적 포트폴리오 탐색
- 방법 2 – scipy 를 이용한 수학적 최적화 (Mathematical Optimization)

- portfolio 생성

- 포트폴리오 수익률 생성 (daily log return)

asset 간의 상관 계수

## Method 1 - Monte Carlo Simulation

- 가능한 allocation 의 조합을 loop 로 반복하여 최적의 조합 (Highest Sharpe ratio) 을 찾는다.


- Sharpe ratio = (기대수익률 - 무위험수익률) / (기대수익률과 무위험수익률 공분산의 표준편차)  

$$SR=\frac{R_p - R_f}{\sigma_p}$$ 



위의 자산을 1/4 씩 동일 비중으로 포트폴리오 구성한 경우 portfolio 의 변동성 계산

**포트폴리오의 연간 covariance**

공분산(covariance)은 두 주식의 수익률 변동이 어떻게 관련되어 있는지 측정하는 지표입니다. 이 경우, 공분산은 제곱 단위으로 측정되므로 연간화할 때 제곱근을 사용하지 않고 거래일 수(일반적으로 252일)를 곱합니다.  

np.dot( ..., weights)는 두 배열의 내적을 계산합니다. 연간 공분산 행렬과 각 주식의 비중(weights)을 곱하여 포트폴리오 전체의 공분산을 계산합니다.

**포트폴리오의 연간 수익률 계산**

각 주식의 평균 로그 수익률에 그 주식이 포트폴리오에서 차지하는 비중을 곱한 후, 이를 모두 더하여(np.sum) 주식 포트폴리오의 일별 예상 수익률을 얻습니다. 그 후, 이 일별 예상 수익률을 연간화하기 위해 252를 곱합니다. 

**포트폴리오의 연간 변동성 계산**

'np.dot(weights.T, cov)'는 가중치 벡터와 공분산 행렬의 내적을 계산하는 것으로, 이 값은 포트폴리오의 분산을 나타냅니다.

이 분산 값에 'np.sqrt' 함수를 적용하여 포트폴리오의 표준편차(또는 변동성)을 계산합니다. 이 값은 포트폴리오의 위험을 나타내며, 값이 클수록 포트폴리오의 위험이 크다는 것을 의미합니다.

포트폴리오의 샤프지수

### 모든 내용을 집합하여 샤프비율 계산 함수 작성
    반환값 - 연간 기대 수익률, 연간 covariance, sharp ratio 반환 

**각 자산의 비중을 무작위로 선택하여 특정 자산 비중에서의 연간 수익률, 변동성, 샤프비율 계산**

주어진 주식 포트폴리오에 대해 각각 랜덤한 가중치를 할당하여 해당 가중치에서의 예상 수익률, 변동성, 샤프 지수를 계산하고, 이를 각각의 배열에 저장하는 과정을 반복적으로 수행합니다. 이렇게 하면 랜덤한 가중치 조합에 대한 포트폴리오 성능을 시뮬레이션하여 최적의 포트폴리오를 탐색하는 데 도움이 됩니다.

- 최적 샤프 비율 (highest sharpe ratio)을 구하고, 그 때의 최적 자산 배분 비율을 구함

최적 Sharpe ratio 일 때의 daily return 과 volatility, 최적 portfolio 비율

- 포트폴리오 조합 및 최적 포트폴리오 시각화

## Method 2 - Scipy 를 이용한 수학적 최적화 (Mathematical Optimization)

수학적 최적화는 주어진 제약 조건 아래에서 목표 함수를 최대화 또는 최소화하는 입력 값(변수)을 찾는 과정입니다. 예를 들어, 포트폴리오 최적화에서 목표 함수는 샤프 지수(포트폴리오의 예상 수익률을 변동성으로 나눈 값)이며, 제약 조건은 포트폴리오의 총 가중치가 1이 되어야 한다는 것입니다.

Scipy는 수치 계산을 위한 파이썬 라이브러리로, 여러 종류의 최적화 알고리즘을 제공합니다. `scipy.optimize` 모듈에서 이 알고리즘들을 사용할 수 있습니다. 이 알고리즘들은 목표 함수와 초기 점, 제약 조건 등을 입력으로 받아 최적의 해를 찾아냅니다.

포트폴리오 최적화에서는 다음과 같은 과정으로 진행됩니다:

1. 목표 함수 정의: 샤프 지수를 최대화하는 것이 목표입니다. 이를 위해 예상 수익률, 변동성 등을 계산하는 함수를 정의합니다.
2. 초기 점 설정: 최적화 알고리즘은 초기 점에서 시작해 최적의 해를 찾아냅니다. 주식의 가중치를 동일하게 설정하는 것이 일반적입니다.
3. 제약 조건 설정: 포트폴리오의 총 가중치가 1이 되도록 제약 조건을 설정합니다.
4. 최적화: Scipy의 최적화 함수를 호출하여 최적의 해를 찾습니다. 이때 목표 함수, 초기 점, 제약 조건 등을 입력으로 제공합니다.

이렇게 하면 최적의 포트폴리오, 즉 샤프 지수를 최대화하는 주식의 가중치를 찾을 수 있습니다. 이를 통해 포트폴리오의 수익률을 최대한 높이고 변동성을 최소화하는 투자 전략을 수립할 수 있습니다.

**Sharpe ratio 의 maximize 를 minimize 함수로 하므로 sharpe ratio * (-1) 을 minimize 하는 trick 을 이용한다.**

- x: objective function을 minimize (Sharpe Ratio를 최대화)하는 x 값 (asset weights)

- MC method로 구한 결과와 수학적 optimize 결과가 유사하면 successful result 이다.

- optimum asset weights 일 때의 return, volatility, sharpe ratio 구하기

## Efficient Frontier (효율적 투자선)

- Efficient Frontier 는 정의된 수준의 위험 또는 주어진 기대수익 수준의 가장 낮은 위험에 대해 가장 높은 수익률을 제공하는 최적 포트폴리오 세트이다. 

- 효율적 전선 아래에 있는 포트폴리오는 동일한 위험 수준에 대해 효율적 전선 상의 포트폴리오 보다 낮은 수익을 제공하므로 최선의 포트폴리오가 아님. 효율적 전선 오른쪽의 포트폴리오는 동일한 수익에 대해 위험 수준이 높으므로 최선의 포트폴리오가 아님.

- possible return이 frontier_y일 때의 volatility를 구한다.  
- optimized_result 의 fun 이 목적함수의 반환값 $\rightarrow$ 즉, volatility 값  
    - fun : Values of objective function  
    - x : 자산 비중

- 모든 return (frontier_y)에 대한 volatility 를 구함

- 모든 possible return 과 volatility를 연결하는 plot 시각화.

### Method 1, 2 를 함께 시각화하여 결과 비교